In [1]:
import numpy as np
from deap import base, creator, tools, algorithms

In [2]:
import sys
sys.path.append('/home/pedroubuntu/TP_CE_2024/env/lib/python3.10/site-packages')

In [3]:
def the_semana(pacientes, ist):
    ''' Explicação:
        Calcula a quantidade de horas de enfermagem necessária em uma semana de acordo com os tipos de pacientes
        Recebe um vetor de quantidade de pacientes por tipo e um índice de segurança (ist)
                    [PCM    PCI     PCAD    PCSI    PCIt]
        pacientes = [40     12      5       2       0   ]
        horas_pd  = [4      6       10      10      18  ]       
        porc_enf  = [0,33   0,33    0,36    0,42    0,52]

        - O numero de horas totais de enfermagem por semana é:
            sum(pacientes * horas_pd)*7*IST
        - A porcentagem de enfermeiros nessas horas é
            porc_enf[argmax(pacientes)]
    ''' 
    horas_pd = [4, 6, 10, 10, 18]
    horas_semana = np.dot(horas_pd, pacientes)*7*ist

    porcentagens = [0.33, 0.33, 0.36, 0.42, 0.52]
    porc_enf = porcentagens[np.argmax(pacientes)]
    return horas_semana, porc_enf 

pacientes = np.array([2, 1, 1, 1, 1])
t1,t2 = the_semana(pacientes, 1.15)
print(t1, t2)

418.59999999999997 0.33


In [4]:
def fitness_funcionarios(individuo, horas_necessarias, porcentagem_enf):
    ''''
    Um indivíduo tem estrutura: [n_enf8h, n_enf12h, n_tec8h, n_tec12h]
    
    Critérios
    1. Somatório de horas é próximo das horas mínimas por semana
    2. Aproximadamente x% das horas são de enfermeiros, x é definido pelo max(PACIENTES_POR_TIPO)
    3. Aproximadamente 2/7 das horas totais são de funcionários de 12h, porque funcionários de 12h não trabalham no fim de semana
    '''
    # Cálculos gerais
    cargas_horarias = np.array([44, 40, 44, 40])
    horas_por_func = individuo * cargas_horarias
    horas_totais = np.sum(horas_por_func)

    # Critério 1: Número de horas é próximo das horas necessárias
    penalidade_1 = np.abs(horas_necessarias - horas_totais)

    # Critério 2: Porcentagem de horas de enfermeiros é próxima do necessário
    penalidade_2 = np.abs(np.sum(horas_por_func[0:2]) - porcentagem_enf*horas_totais)
    
    # Critério 3: Mínimo de funcionários que trabalham 12h
    penalidade_3 = np.abs((horas_por_func[1] + horas_por_func[3]) - 0.29 * horas_totais)
    
    # Combinação dos critérios em um valor de fitness
    valor_fitness = penalidade_1 + penalidade_2 + penalidade_3 

    return valor_fitness, 

individuo = np.array([1,1,1,1])
fit, = fitness_funcionarios(individuo, t1, t2)
print(fit)

310.43999999999994


In [5]:
TAMANHO_POPULACAO = 40
TAMANHO_GENOMA = 4
TAXA_DE_MUTACAO = 0.05
TAXA_DE_CROSSOVER = 0.80
MAX_GERACOES = 1000

In [6]:
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))             # define que a fitness é de minimização  
creator.create('Individual', list, fitness=creator.FitnessMin)          # define a estrutura de um individuo
toolbox = base.Toolbox()        # onde são armazenados 

# Define o range dos valores possíveis de um individuo
max_attr = t1 / 40      # horas_necessarias/40h (pior caso)
toolbox.register('genes', np.random.randint, 0, max_attr + 1)    # um indivíduo so admite valores 0 a max_attr
toolbox.register('individuo', tools.initRepeat, creator.Individual, toolbox.genes, TAMANHO_GENOMA)   # um individuo é uma lista de 4 attr_funcionario 
toolbox.register('populacao', tools.initRepeat, list, toolbox.individuo)    # uma populacao é uma lista de individuos

# Operadores genéticos
toolbox.register("evaluate", fitness_funcionarios)  # funcao de aptidao
toolbox.register("mate", tools.cxTwoPoint)  # crossover
toolbox.register("mutate", tools.mutPolynomialBounded, low=0, up=max_attr, indpb=TAXA_DE_MUTACAO)   # mutação
toolbox.register("select", tools.selTournament, tournsize=3)    # seleção de pais

In [ ]:
def run_genetic_algorithm(horas_necessarias, porc_enf):
    # Configuração do algoritmo genético usando DEAP
    creator.create('FitnessMin', base.Fitness, weights=(-1.0,))             # define que a fitness é de minimização  
    creator.create('Individual', np.ndarray, fitness=creator.FitnessMin)    # define que o individuo é um array np

    # Define os atributos de um individuo 
    toolbox = base.Toolbox()
    max_attr = t1 / 40      # horas_necessarias/40h (pior caso)
    toolbox.register('attr_funcionario', np.random.randint, 0, max_attr + 1)
    # Inicializadores de individuo e populacao
    toolbox.register('individuo', tools.initRepeat, creator.Individual, toolbox.attr_funcionario, TAMANHO_GENOMA)
    toolbox.register("population", tools.initRepeat, list, toolbox.individuo)

    def custom_fitness(individual):
        return fitness_funcionarios(individual, horas_necessarias, porc_enf)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=TAXA_DE_MUTACAO)
    toolbox.register("select", tools.selTournament, tournsize=2 )
    toolbox.register("evaluate", custom_fitness)

    pop = toolbox.population(n=TAMANHO_POPULACAO)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "nevals"] + stats.fields

    for gen in range(MAX_GERACOES):
        offspring = algorithms.varAnd(pop, toolbox, cxpb=TAXA_DE_CROSSOVER, mutpb=TAXA_DE_MUTACAO)
        fits = map(toolbox.evaluate, offspring)
        
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        
        pop = toolbox.select(offspring, k=len(pop))
        record = stats.compile(pop)
        logbook.record(gen=gen, nevals=len(pop), **record)
        hof.update(pop)

    return pop, hof, stats, logbook

pop, hof, stats, logbook = run_genetic_algorithm(t1, t2)
print(pop)

AssertionError: Assigned values have not the same length than fitness weights